# [A Data Science Framework: To Achieve 99% Accuracy](<https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy>)

# Step 1: Define Problems

Predict the survive rate of a passenger on Titanic


# Step 2: Gather Data


Data downloaded from Kaggle located in ./data folder


# Step 3: Prepare Data for Consumption


## 3.1: Import libraries


In [1]:
import sys
print(f'python version: {sys.version}')

import pandas as pd
from tabulate import tabulate # for better df printing
print(f'pandas version: {pd.__version__}')

import matplotlib
print(f'matplotlib version: {matplotlib.__version__}')

import numpy as np
print(f'numpy version: {np.__version__}')

import scipy as sp
print(f'scipy version: {sp.__version__}')

import IPython
from IPython import display
print(f'IPython version: {IPython.__version__}')

import sklearn
print(f'sklearn version: {sklearn.__version__}')

import random
import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('_'*25)

from subprocess import check_output
print(check_output(["ls", "./data"]).decode('utf-8'))



python version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
numpy version: 1.21.5
scipy version: 1.7.3
IPython version: 7.30.1
sklearn version: 1.0.2
_________________________
test.csv
train.csv



## 3.11 Load Data Modelling Libraries


In [2]:
# Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

# Model helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection, model_selection, metrics

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from typing import List, Optional
import plotly.figure_factory as ff

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

## 3.2 Meet and Greet Data


In [3]:
data_raw = pd.read_csv('./data/train.csv')
data_val = pd.read_csv('./data/test.csv')

# create copy of data before play
data_test = data_raw.copy(deep=True)
train_test_pair = [data_test, data_val] # clearn test data set in the same time

# preview data
print(data_raw.info())
data_raw.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
518,519,1,2,"Angle, Mrs. William A (Florence ""Mary"" Agnes H...",female,36.0,1,0,226875,26.0000,NaN,S
476,477,0,2,"Renouf, Mr. Peter Henry",male,34.0,1,0,31027,21.0000,NaN,S
106,107,1,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,NaN,S
725,726,0,3,"Oreskovic, Mr. Luka",male,20.0,0,0,315094,8.6625,NaN,S
338,339,1,3,"Dahl, Mr. Karl Edwart",male,45.0,0,0,7598,8.0500,NaN,S
133,134,1,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.0000,NaN,S
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
762,763,1,3,"Barah, Mr. Hanna Assi",male,20.0,0,0,2663,7.2292,NaN,C
421,422,0,3,"Charters, Mr. David",male,21.0,0,0,A/5. 13032,7.7333,NaN,Q
651,652,1,2,"Doling, Miss. Elsie",female,18.0,0,1,231919,23.0000,NaN,S


### The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting


In [4]:
# check null
print('null in train data: ')
data_test.isnull().sum()

null in train data: 


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
print('null in test/validaton data')
data_val.isnull().sum()

null in test/validaton data


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
data_test.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [7]:
data_test.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

## 3.22 Clean Data

### Handle missing data

In [8]:
# complete missing data
# median for age
for dataset in train_test_pair:
    dataset['Age'].fillna(data_raw['Age'].median(), inplace=True)

    # mode for embarked, mode is object, different from media
    dataset['Embarked'].fillna(data_raw['Embarked'].mode()[0], inplace=True)

    # median for fare
    dataset['Fare'].fillna(data_raw['Fare'].median(), inplace=True)

In [9]:
# delete unrelative/unsiginificant data
# delete columuns
drop_columns = ['PassengerId', 'Ticket', 'Cabin']
data_test.drop(drop_columns, axis=1,inplace=True)

# delte rows

In [10]:
data_test.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [11]:
data_val.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

### Feature Engineering

In [12]:
# Create new features:
for dataset in train_test_pair:
    # family size: SibSp + Parch + self
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

    # isAlone
    dataset['IsAlone'] = dataset['FamilySize'].apply(lambda x: 1 if x == 1 else 0)

    # Title


    # FareBin: separate into 4 groups
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)
    dataset['FareBin'] = dataset['FareBin'].astype(str)

    # AgeBin: split into 5 groups
    dataset['AgeBin'] = pd.cut(dataset['Age'], 5)
    dataset['AgeBin'] = dataset['AgeBin'].astype(str)

print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Name        891 non-null    object 
 3   Sex         891 non-null    object 
 4   Age         891 non-null    float64
 5   SibSp       891 non-null    int64  
 6   Parch       891 non-null    int64  
 7   Fare        891 non-null    float64
 8   Embarked    891 non-null    object 
 9   FamilySize  891 non-null    int64  
 10  IsAlone     891 non-null    int64  
 11  FareBin     891 non-null    object 
 12  AgeBin      891 non-null    object 
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
None


In [13]:
print(data_val.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  FamilySize   418 non-null    int64  
 12  IsAlone      418 non-null    int64  
 13  FareBin      418 non-null    object 
 14  AgeBin       418 non-null    object 
dtypes: float64(2), int64(6), object(7)
memory usage: 49.1+ KB
None


In [14]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Name        891 non-null    object 
 3   Sex         891 non-null    object 
 4   Age         891 non-null    float64
 5   SibSp       891 non-null    int64  
 6   Parch       891 non-null    int64  
 7   Fare        891 non-null    float64
 8   Embarked    891 non-null    object 
 9   FamilySize  891 non-null    int64  
 10  IsAlone     891 non-null    int64  
 11  FareBin     891 non-null    object 
 12  AgeBin      891 non-null    object 
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


## 3.23 Convert Formats

In [15]:

# Code categorical data
label = LabelEncoder()
for dataset in train_test_pair:
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])
    dataset['AgeBin_Code'] = label.fit_transform(dataset['AgeBin'])

# Define target feature
Target = ['Survived']

# Define feature selection
data_test_x = ['Sex','Pclass', 'Embarked','SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone'] #pretty name/values for charts
data_test_x_calc = ['Sex_Code','Pclass', 'Embarked_Code','SibSp', 'Parch', 'Age', 'Fare'] #coded for algorithm calculation
data_test_xy = Target + data_test_x
print(f'Original X Y: {data_test_xy}')

Original X Y: ['Survived', 'Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone']


In [16]:
# define w/bin features
data_test_x_bin = ['Sex_Code','Pclass', 'Embarked_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']
data_test_xy_bin = Target + data_test_x_bin
print(f'Bin X Y: {data_test_xy_bin}')

Bin X Y: ['Survived', 'Sex_Code', 'Pclass', 'Embarked_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']


In [17]:
# define dummy features
data_test_dummy = pd.get_dummies(data_test[data_test_x])
data_test_dummy

,Pclass,SibSp,Parch,Age,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,22.0,7.2500,2,0,0,1,0,0,1
1,1,1,0,38.0,71.2833,2,0,1,0,1,0,0
2,3,0,0,26.0,7.9250,1,1,1,0,0,0,1
3,1,1,0,35.0,53.1000,2,0,1,0,0,0,1
4,3,0,0,35.0,8.0500,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,0,0,27.0,13.0000,1,1,0,1,0,0,1
887,1,0,0,19.0,30.0000,1,1,1,0,0,0,1
888,3,1,2,28.0,23.4500,4,0,1,0,0,0,1
889,1,0,0,26.0,30.0000,1,1,0,1,1,0,0


In [18]:
data_test_x_dummy = data_test_dummy.columns.tolist()
data_test_x_dummy

['Pclass',
 'SibSp',
 'Parch',
 'Age',
 'Fare',
 'FamilySize',
 'IsAlone',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [19]:
data_test_xy_dummy = Target + data_test_x_dummy
data_test_xy_dummy


['Survived',
 'Pclass',
 'SibSp',
 'Parch',
 'Age',
 'Fare',
 'FamilySize',
 'IsAlone',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

## 3.24 Double check cleaned data

In [20]:
print('Train columns with null values: \n', data_test.isnull().sum())
print("-"*10)
print (data_test.info())
print("-"*10)

print('Test/Validation columns with null values: \n', data_val.isnull().sum())
print("-"*10)
print (data_val.info())
print("-"*10)

Train columns with null values: 
 Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         0
FamilySize       0
IsAlone          0
FareBin          0
AgeBin           0
Sex_Code         0
Embarked_Code    0
FareBin_Code     0
AgeBin_Code      0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       891 non-null    int64  
 1   Pclass         891 non-null    int64  
 2   Name           891 non-null    object 
 3   Sex            891 non-null    object 
 4   Age            891 non-null    float64
 5   SibSp          891 non-null    int64  
 6   Parch          891 non-null    int64  
 7   Fare           891 non-null    float64
 8   Embarked       891 non-null    object 
 9   FamilySize     891 non-nu

## 3.25 Split Training and Testing data

In [21]:
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(data_test[data_test_x_calc], data_test[Target], random_state=0)

train1_x_bin, test1_x_bin, train1_y_bin, test1_y_bin = model_selection.train_test_split(data_test[data_test_x_bin], data_test[Target], random_state=0)

train1_x_dummy, test1_x_dummy, train1_y_dummy, test1_y_dummy = model_selection.train_test_split(data_test_dummy[data_test_x_dummy], data_test[Target], random_state=0)

print("Data1 Shape: {}".format(data_test.shape))
print("Train1 Shape: {}".format(train1_x.shape))
print("Test1 Shape: {}".format(test1_x.shape))

Data1 Shape: (891, 17)
Train1 Shape: (668, 7)
Test1 Shape: (223, 7)


# Step 4: EDA

In [22]:
data_test[data_test_x].dtypes

Sex            object
Pclass          int64
Embarked       object
SibSp           int64
Parch           int64
Age           float64
Fare          float64
FamilySize      int64
IsAlone         int64
dtype: object

In [23]:
# data_test['Sex', 'Survived'].groupby(['Sex','Survived'], as_index=False)['Name'].count()


In [24]:
data_test[['Embarked', 'Survived']].groupby('Embarked', as_index=False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [25]:
data_test[['Embarked', 'Survived']][data_test['Survived'] == 1].value_counts()


Embarked  Survived
S         1           219
C         1            93
Q         1            30
dtype: int64

In [26]:
# print info for categorical column
for col in data_test_x:
    if data_test[col].dtype != 'float64':
        print(f'Survival Correlation by : {col}')
        print(data_test[[col, Target[0]]].groupby(col, as_index=False).mean())


Survival Correlation by : Sex
      Sex  Survived
0  female  0.742038
1    male  0.188908
Survival Correlation by : Pclass
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
Survival Correlation by : Embarked
  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009
Survival Correlation by : SibSp
   SibSp  Survived
0      0  0.345395
1      1  0.535885
2      2  0.464286
3      3  0.250000
4      4  0.166667
5      5  0.000000
6      8  0.000000
Survival Correlation by : Parch
   Parch  Survived
0      0  0.343658
1      1  0.550847
2      2  0.500000
3      3  0.600000
4      4  0.000000
5      5  0.200000
6      6  0.000000
Survival Correlation by : FamilySize
   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000
Survival Correlation by : IsAlone

In [27]:
from mc_plot import mc_boxplots_solo
box_plot_cols = ['Fare', 'Age', 'FamilySize']
units = ['$', 'Years', 'Count']
mc_boxplots_solo(data_test, box_plot_cols, units)

In [28]:
from mc_plot import mc_histgram
hist_cols = ['Fare', 'Age','FamilySize']
target = 'Survived' 
mc_histgram(data_test, hist_cols, target)

In [29]:

bar_cols = ['Embarked','Pclass','IsAlone']
from mc_plot import mc_df_bar
mc_df_bar(data_test, bar_cols, 'Survived')

In [30]:
point_cols = ['FareBin', 'AgeBin', 'FamilySize']
# point_cols = ['FamilySize']

mc_df_bar(data_test, point_cols, 'Survived')

In [31]:
from mc_plot import mc_boxplots_xy
cols = ['Fare', 'Age', 'FamilySize']
mc_boxplots_xy(data_test, cols, 'Survived', 'Pclass')
# px.box(data_frame=data_test, x='Pclass', y='Fare', color='Survived')

In [32]:
# TODO bar xy
data_test[['Sex', 'Embarked','Survived']].groupby(['Sex','Embarked'],as_index=False).mean()

,Sex,Embarked,Survived
0,female,C,0.876712
1,female,Q,0.750000
2,female,S,0.692683
3,male,C,0.305263
4,male,Q,0.073171
5,male,S,0.174603


In [33]:
# px.scatter_matrix(data_test,dimensions=['Survived','Pclass','Sex','Age','FamilySize', 'IsAlone','FareBin','AgeBin'])
px.scatter_matrix(data_test,dimensions=['Survived','Pclass','Sex','Age'])

In [34]:
corr = data_test.corr()
corr

,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Sex_Code,Embarked_Code,FareBin_Code,AgeBin_Code
Survived,1.000000,-0.338481,-0.064910,-0.035322,0.081629,0.257307,0.016639,-0.203367,-0.543351,-0.167675,0.101883,-0.043800
Pclass,-0.338481,1.000000,-0.339898,0.083081,0.018443,-0.549500,0.065997,0.135207,0.131900,0.162098,-0.221906,-0.358769
Age,-0.064910,-0.339898,1.000000,-0.233296,-0.172482,0.096688,-0.245619,0.171647,0.081163,-0.018754,0.039764,0.939139
SibSp,-0.035322,0.083081,-0.233296,1.000000,0.414838,0.159651,0.890712,-0.584471,-0.114631,0.068230,0.067507,-0.218747
Parch,0.081629,0.018443,-0.172482,0.414838,1.000000,0.216225,0.783111,-0.583398,-0.245489,0.039798,0.056530,-0.135227
Fare,0.257307,-0.549500,0.096688,0.159651,0.216225,1.000000,0.217138,-0.271832,-0.182333,-0.224719,0.169060,0.123640
FamilySize,0.016639,0.065997,-0.245619,0.890712,0.783111,0.217138,1.000000,-0.690922,-0.200988,0.066516,0.074380,-0.217063
IsAlone,-0.203367,0.135207,0.171647,-0.584471,-0.583398,-0.271832,-0.690922,1.000000,0.303646,0.063532,-0.084823,0.123981
Sex_Code,-0.543351,0.131900,0.081163,-0.114631,-0.245489,-0.182333,-0.200988,0.303646,1.000000,0.108262,-0.093323,0.070220
Embarked_Code,-0.167675,0.162098,-0.018754,0.068230,0.039798,-0.224719,0.066516,0.063532,0.108262,1.000000,0.116838,-0.003402


In [35]:
fig = px.imshow(corr.values, x = corr.columns, y = corr.columns, text_auto=".2f",color_continuous_scale=px.colors.sequential.Blues, aspect='auto')
# z_text = np.around(corr.values, decimals=2)
# fig.update_traces(text=z_text, texttemplate="%{text}")
fig.show()

# Step 5: Model Data

In [36]:
# MLA Selection and Initialization
MLA = [
    # Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),

    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),

    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),

    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),

    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),

    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),

    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    

]

In [37]:
# split frame
cv_split = model_selection.ShuffleSplit(
    n_splits=10, test_size=0.3, train_size=0.6, random_state=0)

# create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Train Accuracy Mean',
               'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD', 'MLA Time']
MLA_compare = pd.DataFrame(columns=MLA_columns)

# table to compare MLA predictions
MLA_predict = data_test[Target]

# index through MLA and save performance to table
row_index = 0
for alg in MLA:
    # set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    # score model with cross validation
    cv_results = model_selection.cross_validate(
        alg, data_test[data_test_x_bin], data_test[Target], cv=cv_split, return_train_score=True)

    MLA_compare.loc[row_index, 'MLA Time'] = \
        cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = \
        cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = \
        cv_results['test_score'].mean()
    # if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = \
        cv_results['test_score'].std(
    )*3  # let's know the worst that can happen!

    # save prediction
    alg.fit(data_test[data_test_x_bin], data_test[Target])
    MLA_predict[MLA_name] = alg.predict(data_test[data_test_x_bin])


    row_index += 1


[18:42:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:42:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:42:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:42:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [38]:
MLA_predict

,Survived,AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier,RandomForestClassifier,GaussianProcessClassifier,LogisticRegressionCV,PassiveAggressiveClassifier,RidgeClassifierCV,...,GaussianNB,KNeighborsClassifier,SVC,NuSVC,LinearSVC,DecisionTreeClassifier,ExtraTreeClassifier,LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,XGBClassifier
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,0,0,0,0,0,1,1,1,...,1,1,1,1,1,0,0,1,1,0
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
887,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
888,0,1,0,0,0,0,0,0,1,1,...,1,0,0,0,1,0,0,1,0,0
889,1,1,0,0,0,0,0,1,1,1,...,1,0,0,0,1,0,0,1,1,0


In [39]:
MLA_compare.sort_values(by=['MLA Test Accuracy Mean'], ascending=False, inplace=True)
MLA_compare

,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time,MLA name
3,NaN,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.859551,0.816791,0.062617,0.057589,GradientBoostingClassifier
2,NaN,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.884082,0.81306,0.065358,0.087117,ExtraTreesClassifier
4,NaN,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.884082,0.812687,0.073199,0.113897,RandomForestClassifier
14,NaN,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.827715,0.812687,0.034611,0.033484,SVC
21,NaN,"{'objective': 'binary:logistic', 'use_label_en...",0.879775,0.811567,0.053976,0.055356,XGBClassifier
15,NaN,"{'break_ties': False, 'cache_size': 200, 'clas...",0.823408,0.810821,0.045622,0.039577,NuSVC
17,NaN,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.884082,0.80709,0.064119,0.002668,DecisionTreeClassifier
5,NaN,"{'copy_X_train': True, 'kernel': None, 'max_it...",0.858052,0.804478,0.059064,0.418226,GaussianProcessClassifier
18,NaN,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.884082,0.802985,0.075392,0.002292,ExtraTreeClassifier
20,NaN,"{'priors': None, 'reg_param': 0.0, 'store_cova...",0.805056,0.799627,0.06736,0.001859,QuadraticDiscriminantAnalysis


In [40]:
# bar plot for accuracy
px.bar(MLA_compare, y = 'MLA name',x = 'MLA Test Accuracy Mean', orientation='h')

## 5.1 Evaluate Model Performance

### How to set up a baseline

First based on probability: 0 or 1 that's 0.5 accuracy
Then account the results frequent occurrence, which is 67.5% people died, we can get 0.675 accuracy

### Create your own model

In [41]:
# TODO coin flip model expected result 0.5

In [42]:
# group by pivot table female
pivot_female = data_test[data_test['Sex'] == 'female'].groupby(
    ['Sex', 'Pclass', 'Embarked'], as_index=False)['Survived'].mean()
pivot_female


,Sex,Pclass,Embarked,Survived
0,female,1,C,0.976744
1,female,1,Q,1.000000
2,female,1,S,0.960000
3,female,2,C,1.000000
4,female,2,Q,1.000000
5,female,2,S,0.910448
6,female,3,C,0.652174
7,female,3,Q,0.727273
8,female,3,S,0.375000


In [43]:
# group by pivot table male
pivot_male = data_test[data_test['Sex'] == 'male'].groupby(
    ['Sex'], as_index=False)['Survived'].mean()
pivot_male

,Sex,Survived
0,male,0.188908


### Build handle made decision tree model

In [44]:
def mytree(df):
    # res = pd.DataFrame(data = {'Predict':[]})
    prediction = []
    for i in df.index:
        row = df.iloc[i]
        if row['Sex'] == 'male':
            prediction.append(0)
        else:
            if row['Pclass'] == 3:
                prediction.append(0) if row['Embarked'] == 'S' \
                    else prediction.append(1)
            else:
                prediction.append(1)

    res = pd.DataFrame(data={'Predict': prediction})

    return res

In [45]:
Tree_Prediction = mytree(data_test)
print(f'Handle made tree model accuracy/precision score: {metrics.accuracy_score(data_test[Target], Tree_Prediction) * 100}. ')

Handle made tree model accuracy/precision score: 81.14478114478115. 


Visualize confusion matrix

In [46]:
metrics.confusion_matrix(data_test[Target], Tree_Prediction)

array([[523,  26],
       [142, 200]])

In [47]:
confusion_matrix = metrics.confusion_matrix(data_test[Target], Tree_Prediction)
class_names = ['Survived', 'Unsurvived']
layout = {
    "title": "Confusion Matrix",
    "xaxis": {"title": "Predicted value"},
    "yaxis": {"title": "Real value"}
}

fig = ff.create_annotated_heatmap(z=confusion_matrix,
                                  x=class_names,
                                  y=class_names,
                                  colorscale=px.colors.sequential.Blues)
fig.show()

In [48]:
confusion_matrix = metrics.confusion_matrix(data_test[Target], Tree_Prediction)
confusion_matrix = confusion_matrix/confusion_matrix.astype(int).sum()
confusion_matrix = np.round(confusion_matrix, 3)
class_names = ['Survived', 'Unsurvived']
layout = {
    "title": "Confusion Matrix",
    "xaxis": {"title": "Predicted value"},
    "yaxis": {"title": "Real value"}
}

fig = ff.create_annotated_heatmap(z=confusion_matrix,
                                  x=class_names,
                                  y=class_names,
                                  colorscale=px.colors.sequential.Blues)
fig.show()


## 5.11 Model Performance with Cross-Validation

# Step 6: Validate and Implement

# Step 7: Optimize and Strategize